📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/media-processing-workshop](https://github.com/mr-pylin/media-processing-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Loading Images](#toc2_)    
- [Projects](#toc3_)    
  - [Exercise 1](#toc3_1_)    
    - [(a)](#toc3_1_1_)    
    - [(b)](#toc3_1_2_)    
    - [(c)](#toc3_1_3_)    
  - [Exercise 2](#toc3_2_)    
    - [(a)](#toc3_2_1_)    
    - [(b)](#toc3_2_2_)    
  - [Exercise 3](#toc3_3_)    
    - [(a)](#toc3_3_1_)    
    - [(b)](#toc3_3_2_)    
    - [(c)](#toc3_3_3_)    
  - [Exercise 4](#toc3_4_)    
    - [(a)](#toc3_4_1_)    
    - [(b)](#toc3_4_2_)    
    - [(c)](#toc3_4_3_)    
  - [Exercise 5](#toc3_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Circle, Rectangle
from numpy.typing import NDArray

# <a id='toc2_'></a>[Loading Images](#toc0_)


In [ ]:
im_1 = cv2.imread("../../../assets/images/dip_3rd/CH02_Fig0222(b)(cameraman).tif", cv2.IMREAD_GRAYSCALE)
im_2 = cv2.imread("../../../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif", cv2.IMREAD_COLOR_RGB)

# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), layout="compressed")
axs[0].imshow(im_1, vmin=0, vmax=255, cmap="gray")
axs[0].set_title("CH02_Fig0222(b)(cameraman).tif")
axs[1].imshow(im_2, vmin=0, vmax=255)
axs[1].set_title("CH06_Fig0638(a)(lenna_RGB).tif")
plt.show()

# <a id='toc3_'></a>[Projects](#toc0_)


## <a id='toc3_1_'></a>[Exercise 1](#toc0_)

- This exercise is based on [**PROJECT 02-01**](https://www.imageprocessingplace.com/DIP-3E/dip3e_student_projects.htm#02-01) from the textbook's student projects page.
- Follow the instructions there to implement the required image processing tasks.


### <a id='toc3_1_1_'></a>[(a)](#toc0_)


In [ ]:
# 10 halftone patterns (3x3 blocks)
halftone_patterns = [
    np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]], dtype=np.uint8),  # black
    np.array([[0, 1, 0], [0, 0, 0], [0, 0, 0]], dtype=np.uint8),
    np.array([[0, 1, 0], [0, 0, 0], [0, 0, 1]], dtype=np.uint8),
    np.array([[1, 1, 0], [0, 0, 0], [0, 0, 1]], dtype=np.uint8),
    np.array([[1, 1, 0], [0, 0, 0], [1, 0, 1]], dtype=np.uint8),
    np.array([[1, 1, 1], [0, 0, 0], [1, 0, 1]], dtype=np.uint8),
    np.array([[1, 1, 1], [0, 0, 1], [1, 0, 1]], dtype=np.uint8),
    np.array([[1, 1, 1], [0, 0, 1], [1, 1, 1]], dtype=np.uint8),
    np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]], dtype=np.uint8),
    np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]], dtype=np.uint8),  # white
]


# plot
fig, axs = plt.subplots(2, 5, figsize=(10, 4), layout="compressed")

for i, ax in enumerate(axs.flat):
    pattern = halftone_patterns[i]
    h, w = pattern.shape

    # set background to white [to simulate paper]
    ax.set_facecolor("white")

    # draw black dots [to simulate black inc]
    for r in range(h):
        for c in range(w):
            if pattern[r, c] == 0:  # black dot
                circ = Circle((c, r), radius=0.5, color="black")
                ax.add_patch(circ)

    for spine in ax.spines.values():
        spine.set_visible(False)

    ax.set(title=f"Pattern {i}", xticks=(), yticks=(), xlim=(-0.5, w - 0.5), ylim=(h - 0.5, -0.5), aspect="equal")

plt.show()

In [ ]:
def scale_to_paper(
    img: NDArray[np.uint8],
    pattern_size: tuple[int, int] = halftone_patterns[0].shape,
    max_width_in: float = 8.5,
    max_height_in: float = 11.0,
    dpi: int = 300,
) -> NDArray[np.uint8]:
    """
    Scale an image so it fits within a physical paper size at the given DPI.

    Parameters:
        img          : Input image (grayscale or color)
        pattern_size : Tuple (pattern_height, pattern_width), size of halftone block in pixels
        max_width_in : Maximum paper width in inches
        max_height_in: Maximum paper height in inches
        dpi          : Dots per inch (printer resolution)

    Returns:
        img_scaled   : Resized image that, after expanding each pixel to a halftone block, fits within the paper
    """

    h, w = img.shape[:2]
    pattern_h, pattern_w = pattern_size

    # compute max pixel dimensions for the paper
    max_w_px = int(max_width_in * dpi)
    max_h_px = int(max_height_in * dpi)

    # compute scaling factor (never upscale)
    scale = min(max_w_px / (w * pattern_w), max_h_px / (h * pattern_h), 1.0)

    if scale < 1.0:
        new_w = int(w * scale)
        new_h = int(h * scale)
        img_scaled = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
    else:
        img_scaled = img.copy()

    return img_scaled

In [ ]:
def halftone(img: NDArray[np.uint8], halftone_patterns: list[NDArray]) -> NDArray[np.uint8]:
    """
    Convert a grayscale or RGB image into a halftone image using provided dot patterns.

    Each pixel in the input image is replaced by a halftone block corresponding to its
    quantized intensity. For RGB images, each channel is processed independently.

    Parameters:
        img               : Input image, either grayscale (HxW) or RGB (HxWx3)
        halftone_patterns : List of 2D arrays representing halftone blocks for each intensity level.
                            All blocks must have the same shape (h_block, w_block).

    Returns:
        Output image with halftone applied. Shape is
        (H*h_block, W*w_block) for grayscale or
        (H*h_block, W*w_block, 3) for RGB.
        Pixel values are scaled to 0–255.
    """

    # determine if the image is grayscale or RGB
    if len(img.shape) == 2:
        channels = [img]
    elif len(img.shape) == 3 and img.shape[2] == 3:
        channels = [img[:, :, ch] for ch in range(3)]
    else:
        raise ValueError("Input image must be grayscale or RGB")

    halftoned_channels = []
    quant_levels = len(halftone_patterns)

    for channel in channels:

        # quantize input intensities
        im_quant = np.floor(channel / 255 * (quant_levels - 1)).astype(np.uint8)

        # Initialize output for this channel
        h, w = channel.shape
        h_block, w_block = halftone_patterns[0].shape
        img_out = np.zeros((h * h_block, w * w_block), dtype=np.uint8)

        # Fill output with patterns
        for r in range(h):
            for c in range(w):
                row_slice = slice(r * h_block, (r + 1) * h_block)
                col_slice = slice(c * w_block, (c + 1) * w_block)
                img_out[row_slice, col_slice] = halftone_patterns[im_quant[r, c]]

        halftoned_channels.append(img_out * 255)

    # stack channels if RGB, otherwise return grayscale
    if len(halftoned_channels) == 1:
        return halftoned_channels[0]
    else:
        return np.stack(halftoned_channels, axis=-1)

### <a id='toc3_1_2_'></a>[(b)](#toc0_)


In [ ]:
# scale and halftone
pattern_h, pattern_w = halftone_patterns[0].shape
wedge = np.tile(np.arange(256, dtype=np.uint8), (256, 1))
wedge_scaled = scale_to_paper(wedge, (pattern_h, pattern_w), 8.5, 11, 300)
wedge_halftone = halftone(wedge_scaled, halftone_patterns)

In [ ]:
# define zoom area in original image
wedge_h_zoom = slice(0, 20)
wedge_w_zoom = slice(100, 180)

# compute corresponding zoom area in halftone image
wedge_ht_h_zoom = slice(wedge_h_zoom.start * pattern_h, wedge_h_zoom.stop * pattern_h)
wedge_ht_w_zoom = slice(wedge_w_zoom.start * pattern_w, wedge_w_zoom.stop * pattern_w)

# rectangle positions and sizes
rect_orig = Rectangle(
    (wedge_w_zoom.start, wedge_h_zoom.start),
    wedge_w_zoom.stop - wedge_w_zoom.start,
    wedge_h_zoom.stop - wedge_h_zoom.start,
    linewidth=2,
    edgecolor="red",
    facecolor="none",
)
rect_ht = Rectangle(
    (wedge_ht_w_zoom.start, wedge_ht_h_zoom.start),
    wedge_ht_w_zoom.stop - wedge_ht_w_zoom.start,
    wedge_ht_h_zoom.stop - wedge_ht_h_zoom.start,
    linewidth=2,
    edgecolor="red",
    facecolor="none",
)

# plot
fig = plt.figure(figsize=(16, 5))
gs = GridSpec(2, 8, figure=fig)
ax_orig = fig.add_subplot(gs[:, :2])
ax_orig.imshow(wedge, cmap="gray", vmin=0, vmax=255)
ax_orig.add_patch(rect_orig)
ax_orig.set_title("Original")
ax_halftone = fig.add_subplot(gs[:, 2:4])
ax_halftone.imshow(wedge_halftone, cmap="gray", vmin=0, vmax=255)
ax_halftone.add_patch(rect_ht)
ax_halftone.set_title("Halftone")
ax_zoom_orig = fig.add_subplot(gs[0, 4:])
ax_zoom_orig.imshow(wedge[wedge_h_zoom, wedge_w_zoom], cmap="gray", vmin=0, vmax=255)
ax_zoom_orig.set_title("Zoom Original")
ax_zoom_orig.axis("off")
ax_zoom_ht = fig.add_subplot(gs[1, 4:])
ax_zoom_ht.imshow(wedge_halftone[wedge_ht_h_zoom, wedge_ht_w_zoom], cmap="gray", vmin=0, vmax=255)
ax_zoom_ht.set_title("Zoom Halftone")
ax_zoom_ht.axis("off")
plt.tight_layout()
plt.show()

### <a id='toc3_1_3_'></a>[(c)](#toc0_)


In [ ]:
# scale and halftone
pattern_h, pattern_w = halftone_patterns[0].shape
im1_scaled = scale_to_paper(im_1, (pattern_h, pattern_w), 8.5, 11, 300)
im1_halftone = halftone(im1_scaled, halftone_patterns)

# define zoom area in original image
im1_h_zoom = slice(45, 75)
im1_w_zoom = slice(105, 135)

# compute corresponding zoom area in halftone image
im1_ht_h_zoom = slice(im1_h_zoom.start * pattern_h, im1_h_zoom.stop * pattern_h)
im1_ht_w_zoom = slice(im1_w_zoom.start * pattern_w, im1_w_zoom.stop * pattern_w)

# rectangle positions and sizes
rect_orig = Rectangle(
    (im1_w_zoom.start, im1_h_zoom.start),
    im1_w_zoom.stop - im1_w_zoom.start,
    im1_h_zoom.stop - im1_h_zoom.start,
    linewidth=2,
    edgecolor="red",
    facecolor="none",
)
rect_ht = Rectangle(
    (im1_ht_w_zoom.start, im1_ht_h_zoom.start),
    im1_ht_w_zoom.stop - im1_ht_w_zoom.start,
    im1_ht_h_zoom.stop - im1_ht_h_zoom.start,
    linewidth=2,
    edgecolor="red",
    facecolor="none",
)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 5), layout="compressed")
axs[0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0].add_patch(rect_orig)
axs[0].set_title("Original")
axs[1].imshow(im1_halftone, cmap="gray", vmin=0, vmax=255)
axs[1].add_patch(rect_ht)
axs[1].set_title("Halftone")
axs[2].imshow(im_1[im1_h_zoom, im1_w_zoom], cmap="gray", vmin=0, vmax=255)
axs[2].set_title("Zoom Original")
axs[3].imshow(im1_halftone[im1_ht_h_zoom, im1_ht_w_zoom], cmap="gray", vmin=0, vmax=255)
axs[3].set_title("Zoom Halftone")
plt.show()

In [ ]:
# scale and halftone
pattern_h, pattern_w = halftone_patterns[0].shape
im2_scaled = scale_to_paper(im_2, (pattern_h, pattern_w), 8.5, 11, 300)
im2_halftone = halftone(im2_scaled, halftone_patterns)

# define zoom area in original image
im2_h_zoom = slice(255, 285)
im2_w_zoom = slice(245, 275)

# compute corresponding zoom area in halftone image
im2_ht_h_zoom = slice(im2_h_zoom.start * pattern_h, im2_h_zoom.stop * pattern_h)
im2_ht_w_zoom = slice(im2_w_zoom.start * pattern_w, im2_w_zoom.stop * pattern_w)

# rectangle positions and sizes
rect_orig = Rectangle(
    (im2_w_zoom.start, im2_h_zoom.start),
    im2_w_zoom.stop - im2_w_zoom.start,
    im2_h_zoom.stop - im2_h_zoom.start,
    linewidth=2,
    edgecolor="white",
    facecolor="none",
)
rect_ht = Rectangle(
    (im2_ht_w_zoom.start, im2_ht_h_zoom.start),
    im2_ht_w_zoom.stop - im2_ht_w_zoom.start,
    im2_ht_h_zoom.stop - im2_ht_h_zoom.start,
    linewidth=2,
    edgecolor="white",
    facecolor="none",
)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 5), layout="compressed")
axs[0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[0].add_patch(rect_orig)
axs[0].set_title("Original")
axs[1].imshow(im2_halftone, cmap="gray", vmin=0, vmax=255)
axs[1].add_patch(rect_ht)
axs[1].set_title("Halftone")
axs[2].imshow(im_2[im2_h_zoom, im2_w_zoom], cmap="gray", vmin=0, vmax=255)
axs[2].set_title("Zoom Original")
axs[3].imshow(im2_halftone[im2_ht_h_zoom, im2_ht_w_zoom], cmap="gray", vmin=0, vmax=255)
axs[3].set_title("Zoom Halftone")
plt.show()

## <a id='toc3_2_'></a>[Exercise 2](#toc0_)

- This exercise is based on [**PROJECT 02-02**](https://www.imageprocessingplace.com/DIP-3E/dip3e_student_projects.htm#02-02) from the textbook's student projects page.
- Follow the instructions there to implement the required image processing tasks.


### <a id='toc3_2_1_'></a>[(a)](#toc0_)


In [ ]:
def reduce_intensity_levels(img: NDArray[np.uint8], target_levels: int) -> NDArray[np.uint8]:
    """
    Reduce the number of intensity levels in an image.

    Parameters:
        img           : input image (grayscale)
        target_levels : desired number of intensity levels (must be power of 2)

    Returns:
        img_quantized : processed image with reduced intensity levels
    """

    # validate that target_levels is a power of 2
    if target_levels <= 0 or (target_levels & (target_levels - 1)) != 0:
        raise ValueError("Target levels must be a positive power of 2")

    # validate that target_levels is less than or equal to 256
    if target_levels > 256:
        raise ValueError("Target levels must be less than or equal to 256")

    # quantize the image
    img_quantized = np.round((img / 255) * (target_levels - 1)) / (target_levels - 1) * 255
    img_quantized = np.clip(img_quantized, 0, 255).astype(np.uint8)

    return img_quantized

### <a id='toc3_2_2_'></a>[(b)](#toc0_)


In [ ]:
im1_scale_1 = reduce_intensity_levels(im_1, target_levels=128)
im1_scale_2 = reduce_intensity_levels(im_1, target_levels=32)
im1_scale_3 = reduce_intensity_levels(im_1, target_levels=8)
im1_scale_4 = reduce_intensity_levels(im_1, target_levels=2)

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im1_scale_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("levels=128")
axs[2].imshow(im1_scale_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("levels=32")
axs[3].imshow(im1_scale_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("levels=8")
axs[4].imshow(im1_scale_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("levels=2")
plt.show()

In [ ]:
im2_scale_1 = reduce_intensity_levels(im_2, target_levels=128)
im2_scale_2 = reduce_intensity_levels(im_2, target_levels=32)
im2_scale_3 = reduce_intensity_levels(im_2, target_levels=8)
im2_scale_4 = reduce_intensity_levels(im_2, target_levels=2)

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im2_scale_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("levels=128")
axs[2].imshow(im2_scale_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("levels=32")
axs[3].imshow(im2_scale_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("levels=8")
axs[4].imshow(im2_scale_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("levels=2")
plt.show()

## <a id='toc3_3_'></a>[Exercise 3](#toc0_)

- This exercise is based on [**PROJECT 02-03**](https://www.imageprocessingplace.com/DIP-3E/dip3e_student_projects.htm#02-03) from the textbook's student projects page.
- Follow the instructions there to implement the required image processing tasks.


### <a id='toc3_3_1_'></a>[(a)](#toc0_)


In [ ]:
def zoom_shrink_pixel_replication(img: NDArray[np.uint8], factor: int, mode: str = "zoom") -> NDArray[np.uint8]:
    """
    Zoom or shrink an image using pixel replication.

    Parameters:
        img    : input image (grayscale or RGB)
        factor : integer zoom/shrink factor (>0)
        mode   : 'zoom' to enlarge, 'shrink' to reduce size

    Returns:
        Processed image
    """

    if factor <= 0 or not isinstance(factor, int):
        raise ValueError("Factor must be a positive integer")

    if mode not in ["zoom", "shrink"]:
        raise ValueError("Mode must be 'zoom' or 'shrink'")

    if mode == "zoom":
        return np.repeat(np.repeat(img, factor, axis=0), factor, axis=1)

    elif mode == "shrink":
        return img[::factor, ::factor] if img.ndim == 2 else img[::factor, ::factor, :]

### <a id='toc3_3_2_'></a>[(b)](#toc0_)


In [ ]:
im1_shrink_1 = zoom_shrink_pixel_replication(im_1, factor=2, mode="shrink")
im1_shrink_2 = zoom_shrink_pixel_replication(im_1, factor=4, mode="shrink")
im1_shrink_3 = zoom_shrink_pixel_replication(im_1, factor=8, mode="shrink")
im1_shrink_4 = zoom_shrink_pixel_replication(im_1, factor=16, mode="shrink")

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im1_shrink_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("factor=2")
axs[2].imshow(im1_shrink_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("factor=4")
axs[3].imshow(im1_shrink_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("factor=8")
axs[4].imshow(im1_shrink_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("factor=16")
plt.show()

In [ ]:
im2_shrink_1 = zoom_shrink_pixel_replication(im_2, factor=2, mode="shrink")
im2_shrink_2 = zoom_shrink_pixel_replication(im_2, factor=4, mode="shrink")
im2_shrink_3 = zoom_shrink_pixel_replication(im_2, factor=8, mode="shrink")
im2_shrink_4 = zoom_shrink_pixel_replication(im_2, factor=16, mode="shrink")

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im2_shrink_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("factor=2")
axs[2].imshow(im2_shrink_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("factor=4")
axs[3].imshow(im2_shrink_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("factor=8")
axs[4].imshow(im2_shrink_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("factor=16")
plt.show()

### <a id='toc3_3_3_'></a>[(c)](#toc0_)


In [ ]:
im1_zoom_1 = zoom_shrink_pixel_replication(im1_shrink_1, factor=2, mode="zoom")
im1_zoom_2 = zoom_shrink_pixel_replication(im1_shrink_2, factor=4, mode="zoom")
im1_zoom_3 = zoom_shrink_pixel_replication(im1_shrink_3, factor=8, mode="zoom")
im1_zoom_4 = zoom_shrink_pixel_replication(im1_shrink_4, factor=16, mode="zoom")

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im1_zoom_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("factor=2")
axs[2].imshow(im1_zoom_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("factor=4")
axs[3].imshow(im1_zoom_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("factor=8")
axs[4].imshow(im1_zoom_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("factor=16")
plt.show()

In [ ]:
im2_zoom_1 = zoom_shrink_pixel_replication(im2_shrink_1, factor=2, mode="zoom")
im2_zoom_2 = zoom_shrink_pixel_replication(im2_shrink_2, factor=4, mode="zoom")
im2_zoom_3 = zoom_shrink_pixel_replication(im2_shrink_3, factor=8, mode="zoom")
im2_zoom_4 = zoom_shrink_pixel_replication(im2_shrink_4, factor=16, mode="zoom")

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im2_zoom_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("factor=2")
axs[2].imshow(im2_zoom_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("factor=4")
axs[3].imshow(im2_zoom_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("factor=8")
axs[4].imshow(im2_zoom_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("factor=16")
plt.show()

## <a id='toc3_4_'></a>[Exercise 4](#toc0_)

- This exercise is based on [**PROJECT 02-04**](https://www.imageprocessingplace.com/DIP-3E/dip3e_student_projects.htm#02-04) from the textbook's student projects page.
- Follow the instructions there to implement the required image processing tasks.


### <a id='toc3_4_1_'></a>[(a)](#toc0_)


In [ ]:
def bilinear_resize(img: NDArray[np.uint8], input_dpi: float, target_dpi: float) -> NDArray[np.uint8]:
    """
    Zoom or shrink an image using bilinear interpolation based on DPI.

    Parameters:
        img        : Input image (grayscale or RGB)
        input_dpi  : DPI of the input image
        target_dpi : Desired DPI of the output image

    Returns:
        Resized image
    """

    if input_dpi <= 0 or target_dpi <= 0:
        raise ValueError("DPI values must be positive")

    # compute scaling factor
    scale = target_dpi / input_dpi
    h_in, w_in = img.shape[:2]
    h_out, w_out = int(h_in * scale), int(w_in * scale)

    # generate coordinates in the output image
    row_idx = (np.arange(h_out) + 0.5) / scale - 0.5
    col_idx = (np.arange(w_out) + 0.5) / scale - 0.5

    row_idx = np.clip(row_idx, 0, h_in - 1)
    col_idx = np.clip(col_idx, 0, w_in - 1)

    r_floor = np.floor(row_idx).astype(int)
    r_ceil = np.ceil(row_idx).astype(int)
    c_floor = np.floor(col_idx).astype(int)
    c_ceil = np.ceil(col_idx).astype(int)

    r_weight = row_idx - r_floor
    c_weight = col_idx - c_floor

    if img.ndim == 2:
        out_img = np.zeros((h_out, w_out), dtype=np.float32)
        for i, (rf, rcw) in enumerate(zip(r_floor, r_weight)):
            for j, (cf, ccw) in enumerate(zip(c_floor, c_weight)):
                top_left = img[rf, cf]
                top_right = img[rf, c_ceil[j]]
                bottom_left = img[r_ceil[i], cf]
                bottom_right = img[r_ceil[i], c_ceil[j]]

                top = top_left * (1 - ccw) + top_right * ccw
                bottom = bottom_left * (1 - ccw) + bottom_right * ccw
                out_img[i, j] = top * (1 - rcw) + bottom * rcw
    else:
        out_img = np.zeros((h_out, w_out, img.shape[2]), dtype=np.float32)
        for i, (rf, rcw) in enumerate(zip(r_floor, r_weight)):
            for j, (cf, ccw) in enumerate(zip(c_floor, c_weight)):
                for k in range(img.shape[2]):
                    top_left = img[rf, cf, k]
                    top_right = img[rf, c_ceil[j], k]
                    bottom_left = img[r_ceil[i], cf, k]
                    bottom_right = img[r_ceil[i], c_ceil[j], k]

                    top = top_left * (1 - ccw) + top_right * ccw
                    bottom = bottom_left * (1 - ccw) + bottom_right * ccw
                    out_img[i, j, k] = top * (1 - rcw) + bottom * rcw

    return np.clip(out_img, 0, 255).astype(np.uint8)

### <a id='toc3_4_2_'></a>[(b)](#toc0_)


In [ ]:
im1_shrink_1 = bilinear_resize(im_1, input_dpi=300, target_dpi=200)
im1_shrink_2 = bilinear_resize(im_1, input_dpi=300, target_dpi=150)
im1_shrink_3 = bilinear_resize(im_1, input_dpi=300, target_dpi=100)
im1_shrink_4 = bilinear_resize(im_1, input_dpi=300, target_dpi=50)

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im1_shrink_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("factor=2")
axs[2].imshow(im1_shrink_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("factor=4")
axs[3].imshow(im1_shrink_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("factor=8")
axs[4].imshow(im1_shrink_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("factor=16")
plt.show()

In [ ]:
im2_shrink_1 = bilinear_resize(im_2, input_dpi=300, target_dpi=200)
im2_shrink_2 = bilinear_resize(im_2, input_dpi=300, target_dpi=150)
im2_shrink_3 = bilinear_resize(im_2, input_dpi=300, target_dpi=100)
im2_shrink_4 = bilinear_resize(im_2, input_dpi=300, target_dpi=50)

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im2_shrink_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("factor=2")
axs[2].imshow(im2_shrink_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("factor=4")
axs[3].imshow(im2_shrink_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("factor=8")
axs[4].imshow(im2_shrink_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("factor=16")
plt.show()

### <a id='toc3_4_3_'></a>[(c)](#toc0_)


In [ ]:
im1_zoom_1 = bilinear_resize(im1_shrink_1, input_dpi=200, target_dpi=300)
im1_zoom_2 = bilinear_resize(im1_shrink_2, input_dpi=150, target_dpi=300)
im1_zoom_3 = bilinear_resize(im1_shrink_3, input_dpi=100, target_dpi=300)
im1_zoom_4 = bilinear_resize(im1_shrink_4, input_dpi=50, target_dpi=300)

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im1_zoom_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("factor=2")
axs[2].imshow(im1_zoom_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("factor=4")
axs[3].imshow(im1_zoom_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("factor=8")
axs[4].imshow(im1_zoom_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("factor=16")
plt.show()

In [ ]:
im2_zoom_1 = bilinear_resize(im2_shrink_1, input_dpi=200, target_dpi=300)
im2_zoom_2 = bilinear_resize(im2_shrink_2, input_dpi=150, target_dpi=300)
im2_zoom_3 = bilinear_resize(im2_shrink_3, input_dpi=100, target_dpi=300)
im2_zoom_4 = bilinear_resize(im2_shrink_4, input_dpi=50, target_dpi=300)

# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original")
axs[1].imshow(im2_zoom_1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("factor=2")
axs[2].imshow(im2_zoom_2, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("factor=4")
axs[3].imshow(im2_zoom_3, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("factor=8")
axs[4].imshow(im2_zoom_4, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("factor=16")
plt.show()

## <a id='toc3_5_'></a>[Exercise 5](#toc0_)

- This exercise is based on [**PROJECT 02-05**](https://www.imageprocessingplace.com/DIP-3E/dip3e_student_projects.htm#02-05) from the textbook's student projects page.
- Follow the instructions there to implement the required image processing tasks.


In [ ]:
def image_arithmetic(
    img1: NDArray[np.uint8],
    img2: NDArray[np.uint8] | float | int,
    operation: str,
) -> NDArray[np.uint8]:
    """
    Perform arithmetic operations between two images or an image and a scalar.

    Parameters:
        img1      : First input image (grayscale or RGB)
        img2      : Second input image or scalar
        operation : 'add', 'subtract', 'multiply', 'divide'

    Returns:
        Resulting image as uint8 with values clipped to [0, 255]
    """

    if operation not in ["add", "subtract", "multiply", "divide"]:
        raise ValueError("Operation must be 'add', 'subtract', 'multiply', or 'divide'")

    # convert inputs to float for safe arithmetic
    img1_f = img1.astype(np.float32)

    if isinstance(img2, (int, float)):
        img2_f = float(img2)
    else:
        img2_f = img2.astype(np.float32)
        if img1.shape != img2.shape:
            raise ValueError("Image shapes must match for image-image operations")

    if operation == "add":
        result = img1_f + img2_f

    elif operation == "subtract":
        result = img1_f - img2_f

    elif operation == "multiply":
        result = img1_f * img2_f

    elif operation == "divide":
        # avoid division by zero
        if isinstance(img2_f, (int, float)):
            if img2_f == 0:
                raise ValueError("Cannot divide by zero scalar")
        else:
            img2_f = np.where(img2_f == 0, 1e-6, img2_f)
        result = img1_f / img2_f

    # clip to [0, 255] and convert to uint8
    result = np.clip(result, 0, 255).astype(np.uint8)
    return result

In [ ]:
im1 = im_1.copy()
im2 = im_2.copy()[::2, ::2].mean(axis=2)

# perform arithmetic operations
arithmetic_1 = image_arithmetic(im1, im2, operation="add")
arithmetic_2 = image_arithmetic(im1, im2, operation="subtract")
arithmetic_3 = image_arithmetic(im1, im2, operation="multiply")
arithmetic_4 = image_arithmetic(im1, 2, operation="multiply")
arithmetic_5 = image_arithmetic(im1, im2, operation="divide")

In [ ]:
# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 5), layout="compressed")
axs[0].imshow(arithmetic_1, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("arithmetic_1")
axs[1].imshow(arithmetic_2, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("arithmetic_2")
axs[2].imshow(arithmetic_3, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("arithmetic_3")
axs[3].imshow(arithmetic_4, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("arithmetic_4")
axs[4].imshow(arithmetic_5, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("arithmetic_5")
plt.show()